<a href="https://colab.research.google.com/github/vveitch/causality-tutorials/blob/main/difference_in_differences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Difference-in-Differences Estimation Tutorial

A short example on how to estimate the difference-in-differences ATT with 2 period panel data using using machine learning methods.

Data from this paper: https://www.aeaweb.org/articles?id=10.1257/000282804322970733

In brief: following a terrorist attack on a synagogue in Buenos Aires, additional police officers were stationed on blocks containing Jewish institutions. This provides a natural experiment for the effect of policing on deterring crime. The data includes the number of car thefts in many city blocks the months before and after the increase in policing. Comparing the change in thefts for blocks with Jewish institutions (hence, increased police) to the other blocks gives a measurement. However, blocks with Jewish institutions may differ in significant ways---e.g., they may tend to be better educated or located in certain neighbourhoods. We want to use machine learning methods to control for such potential issues. 

In [197]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os

In [198]:
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

##Load and Format Data

We reformat the data so that there is an "outcome" column equal to the difference in car thefts after and before the time period, a "treatment" column indictaing the presence of a jewish institute, and "confounders" denoting variables that may differ between jewish and non-jewish blocks, and which may also affect the change in crime rate. 

After doing this formatting, the estimation procedure is identical to computing the ATT with a regression adjustment

In [199]:
panel = pd.read_csv('https://raw.githubusercontent.com/vveitch/causality-tutorials/main/data/ditella-crime-2004/DiTella_crime.csv')


In [200]:
panel.head()

,block,neighbourhood,street,jewish_insitute,public_institution,gas_station,bank,car_thefts,month,education,employment_rate
0,870.0,Once,Cordoba,0.0,1.0,0.0,0.0,0.0,4.0,10.846611,0.949495
1,870.0,Once,Cordoba,0.0,1.0,0.0,0.0,0.0,5.0,10.846611,0.949495
2,870.0,Once,Cordoba,0.0,1.0,0.0,0.0,0.0,6.0,10.846611,0.949495
3,870.0,Once,Cordoba,0.0,1.0,0.0,0.0,0.0,7.0,10.846611,0.949495
4,870.0,Once,Cordoba,0.0,1.0,0.0,0.0,0.0,8.0,10.846611,0.949495


In [201]:
# Terrorist attack occurred in July 18, and increased police presence begins July 25. Data before this is before period, and after is after period 
first_period = panel['month'].isin([4., 5., 6., 71.])
panel['first_period']=first_period

In [202]:
# code neighbourhood as integer for later convenience
panel['neighbourhood']=panel['neighbourhood'].astype('category').cat.codes

In [203]:
# We need to reduce the multiple before and after months in some fashion
# There is not a clear canonical way to do this, but an average seems reasonable
panel = panel.groupby(['block', 'first_period']).mean()
panel = panel.reset_index(level='first_period')
panel

,first_period,neighbourhood,jewish_insitute,public_institution,gas_station,bank,car_thefts,month,education,employment_rate
block,,,,,,,,,,
1.0,False,0,0.0,0.0,0.0,0.0,0.000000,25.25,11.919889,0.926594
1.0,True,0,0.0,0.0,0.0,0.0,0.000000,5.00,11.919889,0.926594
2.0,False,0,0.0,0.0,0.0,0.0,0.156250,25.25,11.919889,0.926594
2.0,True,0,0.0,0.0,0.0,0.0,0.000000,5.00,11.919889,0.926594
3.0,False,0,0.0,0.0,0.0,0.0,0.031250,25.25,11.919889,0.926594
...,...,...,...,...,...,...,...,...,...,...
874.0,True,1,0.0,0.0,0.0,0.0,0.000000,5.00,10.898485,0.939759
875.0,False,1,0.0,0.0,0.0,0.0,0.000000,25.25,10.898485,0.939759
875.0,True,1,0.0,0.0,0.0,0.0,0.083333,5.00,10.898485,0.939759


In [204]:
# now create a version of the data w/ "outcome" = after - before thefts, and 
compact_df = panel[~panel['first_period']]
car_thefts = panel['car_thefts'].values
compact_df['Y1-Y0']=car_thefts[~panel['first_period']] - car_thefts[panel['first_period']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [205]:
# format this in a manner sympatico with ATT estimation
compact_df = compact_df.reset_index()

outcome = compact_df['Y1-Y0']
treatment = compact_df['jewish_insitute']
confounders = compact_df[['neighbourhood','public_institution', 'gas_station', 'bank', 'education', 'employment_rate']]

In [206]:
# finally, do some light data cleaning
treatment=treatment.astype(int)

# scale continuous covariates
cont_vars = ['education', 'employment_rate']
confounders[cont_vars] = preprocessing.scale(confounders[cont_vars])



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


## Specify Nuisance Function Models

The next step is to specify models for the conditional expected outcome and propensity score

In [207]:
# specify a model for the conditional expected outcome

# TODO(victorveitch) the covariates have basically no predictive power, replace this example with something better

# make a function that returns a sklearn model for later use in k-folding
def make_Q_model():
  # return LinearRegression()
 return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=100, max_depth=2)
Q_model = make_Q_model()

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict(X_test)

test_mse=mean_squared_error(y_pred, y_test)
print(f"Test MSE of fit model {test_mse}") 
baseline_mse=mean_squared_error(y_train.mean()*np.ones_like(y_test), y_test)
print(f"Test MSE of no-covariate model {baseline_mse}")

Test MSE of fit model 0.027801530904389606
Test MSE of no-covariate model 0.028564516759592096


In [208]:
# specify a model for the propensity score

def make_g_model():
#  return LogisticRegression(max_iter=1000)
  return RandomForestClassifier(n_estimators=100, max_depth=2)

g_model = make_g_model()
# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_train, X_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)
g_model.fit(X_train, a_train)
a_pred = g_model.predict_proba(X_test)[:,1]

test_ce=log_loss(a_test, a_pred)
print(f"Test CE of fit model {test_ce}") 
baseline_ce=log_loss(a_test, a_train.mean()*np.ones_like(a_test))
print(f"Test CE of no-covariate model {baseline_ce}")

Test CE of fit model 0.1597166570168377
Test CE of no-covariate model 0.16733990853941555


## Use cross fitting to get get predicted outcomes and propensity scores for each unit

In [209]:
# helper functions to implement the cross fitting

def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
      X_train = X.loc[train_index]
      A_train = A.loc[train_index]
      g = make_model()
      g.fit(X_train, A_train)

      # get predictions for split
      predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]

    assert np.isnan(predictions).sum() == 0
    return predictions


def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
      kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
      X_train = X_w_treatment.loc[train_index]
      y_train = y.loc[train_index]
      q = make_model()
      q.fit(X_train, y_train)

      if output_type =='binary':
        predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
        predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
      elif output_type == 'continuous':
        predictions0[test_index] = q.predict(X0.loc[test_index])
        predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [210]:
g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=10)

In [211]:
Q0,Q1=outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, n_splits=10, output_type="continuous")

In [212]:
data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})
data_and_nuisance_estimates.head()

,g,Q0,Q1,A,Y
0,0.027920,-0.065413,-0.133397,0,0.000000
1,0.027276,-0.070393,-0.118878,0,0.156250
2,0.028456,-0.076041,-0.142226,0,-0.302083
3,0.028456,-0.065413,-0.133397,0,0.062500
4,0.025655,-0.020747,-0.085449,0,0.062500


## Combine predicted values and data into estimate of ATT

In [213]:
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
  """
  # Double ML estimator for the ATT
  This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
  """

  if prob_t is None:
    prob_t = A.mean() # estimate marginal probability of treatment

  tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
  scores = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
  n = Y.shape[0] # number of observations
  std_hat = np.std(scores) / np.sqrt(n)

  return tau_hat, std_hat


In [214]:
tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is -0.0777691984649497 pm 0.05810535308191231


In [215]:
# for comparison, the point estimate without any covariate correction
outcome[treatment==1].mean()-outcome[treatment==0].mean()

-0.06683773314434818